In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime

In [16]:
from google.cloud import bigquery

In [17]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the environment variable
credentials_file = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")

In [25]:
# create dataset

client = bigquery.Client()

dataset_id = "{}.headline_dataset".format(client.project)
dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"
dataset = client.create_dataset(dataset, timeout=30)  # Make an API request.
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

Created dataset sharedgcpproject.headline_dataset


In [28]:
dataUpload = pd.DataFrame()
def scraper(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    if url[24:30] == "search":
        blocks = soup.find_all("div", class_="xrnccd")
    elif url[24:30] == "topics":
        blocks = soup.find_all("c-wiz", class_="PO9Zff Ccj79 kUVvS")
    
    HL = []
    DATE = []
    TIME = []
    LINK = []
    PB = []
    for i in range(len(blocks)):
        if url[24:30] == "search":
            headline = blocks[i].find("a", class_="DY5T1d RZIKme").get_text()
            link = blocks[i].find("a", class_="DY5T1d RZIKme").get("href")
            date_and_time = blocks[i].find("time", class_="WW6dff uQIVzc Sksgp slhocf")
            publisher = soup.find("a", class_="wEwyrc AVN2gc WfKKme").get_text()
        elif url[24:30] == "topics":
            headline = blocks[i].find("a", class_="WwrzSb").get("aria-label")
            link = blocks[i].find("a", class_="WwrzSb").get("href")
            date_and_time = blocks[i].find("time", class_="hvbAAd")
            publisher = soup.find("span", class_="vr1PYe").get_text()

        HL.append(headline)
        LINK.append("https://news.google.com/"+link)
        PB.append(publisher)
        DATE.append(re.match("^([^T]+)", date_and_time.get("datetime")).group(0))
        TIME.append(re.search(r'(?<=T)(.*)(?=Z)', date_and_time.get("datetime")).group(1))

    if url[24:30] == "search":
        search_data = pd.DataFrame({"Headline": HL, "Date": DATE, "Time": TIME, "link": LINK, "Publisher": PB})
        # still need to adjust the file name according to the url
        search_data.to_csv(str(datetime.today().date())+"_DSdata.csv")
        print(str(len(search_data))+" headlines scraped from search!")
        
        
    else:
        data = pd.DataFrame({"Headline": HL, "Date": DATE, "Time": TIME, "link": LINK, "Publisher": PB})
        today = str(datetime.today().date())
        data.to_csv(today+"_USdata.csv") 
        print(str(len(data))+" headlines scraped from topics!")
    
    
    table_id = 'headline_dataset.new_table'
    
    job_config = bigquery.LoadJobConfig(schema=[bigquery.SchemaField("Headline", "STRING"),])
    job = client.load_table_from_dataframe(search_data, table_id, job_config=job_config)
        

In [29]:
# url for the US topic google news
url = "https://news.google.com/topics/CAAqIggKIhxDQkFTRHdvSkwyMHZNRGxqTjNjd0VnSmxiaWdBUAE?hl=en-US&gl=US&ceid=US%3Aen"
# url for the Data Science search results
url = "https://news.google.com/search?q=data%20science&hl=en-US&gl=US&ceid=US%3Aen"

scraper(url)

100 headlines scraped from search!


""
